In [34]:
import pandas as pd
from datetime import datetime
import math


# Pricing Models: Pricing Natural Gas Contracts

Objective: The objective is to create a script that prices a natural gas contract by calculating the contract value based on the difference between the expected selling price and buying price, deducting costs such as storage, injection/withdrawal, and transportation.

Task: Write a function that takes these inputs and gives back the value of the contract. 

Notes about the data: 
- Using the forecasted values from the **Forecasting Natural Gas Market Price**

The input parameters that should be taken into account for pricing are:

- Injection dates. 
- Withdrawal dates.
- The prices at which the commodity can be purchased/sold on those dates.
- The rate at which the gas can be injected/withdrawn.
- The maximum volume that can be stored.
- Storage costs.


Assumptions:
- No transport delay
- Interest rates are zero
- Market holidays, weekends, and bank holidays need not be accounted for.

In [37]:
forecasted_prices = pd.read_csv('forecast_Nat_Gas.csv')

forecasted_prices

,Unnamed: 0,Dates,Prices
0,0,2024-10-31,12.125723
1,1,2024-11-30,12.588711
2,2,2024-12-31,12.934792
3,3,2025-01-31,13.039962
4,4,2025-02-28,12.761522
5,5,2025-03-31,13.052298
6,6,2025-04-30,12.494462
7,7,2025-05-31,11.979337
8,8,2025-06-30,11.898213
9,9,2025-07-31,12.174310


In [ ]:
class NaturalGasStorageContract:
    def __init__(self, injection_dates, injection_volumes, withdrawal_dates, withdrawal_volumes, 
                 purchase_prices, selling_prices, injection_withdrawal_rate, max_storage, 
                 storage_costs_per_unit):
        self.injection_dates = self.convert_to_date(injection_dates)
        self.withdrawal_dates = self.convert_to_date(withdrawal_dates)
        self.injection_volumes = injection_volumes
        self.withdrawal_volumes = withdrawal_volumes
        self.purchase_prices = purchase_prices
        self.selling_prices = selling_prices
        self.injection_withdrawal_rate = injection_withdrawal_rate
        self.max_storage = max_storage
        self.storage_costs_per_unit = storage_costs_per_unit
        self.forecasted_prices = self.load_forecasted_prices()

    """
    Calculates the value of a natural gas contract.

    Parameters:
    - injection_dates (list of str): Dates for gas injection
    - withdrawal_dates (list of str): Dates for gas withdrawal
    - purchase_prices (list of float): Prices for purchasing gas on injection dates
    - selling_prices (list of float): Prices for selling gas on withdrawal dates
    - injection_withdrawal_rate (float): Rate of gas injection/withdrawal per day (units)
    - max_storage (float): Maximum volume of gas that can be stored (units)
    - storage_cost_per_unit (float): Cost of storing one unit of gas
    """

    def convert_to_date(self, dates):
        """Convert list of date strings to datetime.date objects and sort them."""
        return sorted([pd.to_datetime(date, format="%Y-%m-%d").date() for date in dates])

    def load_forecasted_prices(self):
        """Load forecasted prices from CSV file from Task 1."""
        try:
            forecasted_prices = pd.read_csv('forecast_Nat_Gas.csv')

            # Check if 'Dates' column exists and handle it
            if 'Dates' not in forecasted_prices.columns:
                raise ValueError("CSV file must contain a 'Dates' column.")

            # Convert 'Dates' column to datetime.date and set it as index
            forecasted_prices['Dates'] = pd.to_datetime(forecasted_prices['Dates']).dt.date
            forecasted_prices.set_index('Dates', inplace=True)
            return forecasted_prices
        except FileNotFoundError:
            raise FileNotFoundError("The file 'forecast_Nat_Gas.csv' could not be found.")
        except ValueError as e:
            raise ValueError(f"Error with the data in the CSV file: {e}")
        except Exception as e:
            raise Exception(f"Error loading forecasted prices: {e}")

    def calculate_injection_cost(self, inject_vol, inject_price):
        """Calculate the cost of injecting gas."""
        return inject_vol * inject_price + inject_vol * self.injection_withdrawal_rate

    def calculate_withdrawal_value(self, withdraw_vol, withdraw_price):
        """Calculate the value obtained from withdrawing gas."""
        return withdraw_vol * withdraw_price - withdraw_vol * self.injection_withdrawal_rate

    def check_storage_capacity(self, current_volume, inject_vol):
        """Check if there is enough storage capacity for injection."""
        return current_volume + inject_vol <= self.max_storage

    def calculate_storage_cost(self, contract_duration_days):
        """Calculate the storage cost for the entire duration of the contract."""
        storage_duration_months = math.ceil(contract_duration_days / 30) # Assuming 30-day months
        return storage_duration_months * self.storage_costs_per_unit

    def compute_net_contract_value(self):
        """Compute the net value of the contract."""
        volume = 0.0  # Current stored volume
        total_buy_cost = 0.0
        total_sell_value = 0.0

        # Combine and sort all relevant dates
        relevant_dates = sorted(set(self.injection_dates).union(set(self.withdrawal_dates)))

        for current_date in relevant_dates:
            # Handle injections
            if current_date in self.injection_dates:
                index = self.injection_dates.index(current_date)
                inject_vol = self.injection_volumes[index]

                # Try fetching price for injection, and handle missing dates
                try:
                    inject_price = self.forecasted_prices.loc[current_date].iloc[0]
                except KeyError:
                    print(f"Warning: No forecasted price available for injection on {current_date}. Skipping.")
                    continue

                # Check if there's enough storage capacity for the injection
                if self.check_storage_capacity(volume, inject_vol):
                    volume += inject_vol
                    total_buy_cost += self.calculate_injection_cost(inject_vol, inject_price)
                    print(f"Injected {inject_vol} units of gas on {current_date} at price {inject_price:.2f}")
                else:
                    print(f"Insufficient storage capacity on {current_date} for {inject_vol} units")

            # Handle withdrawals
            if current_date in self.withdrawal_dates:
                index = self.withdrawal_dates.index(current_date)
                withdraw_vol = self.withdrawal_volumes[index]

                # Try fetching price for withdrawal, and handle missing dates
                try:
                    withdraw_price = self.forecasted_prices.loc[current_date].iloc[0]
                except KeyError:
                    print(f"Warning: No forecasted price available for withdrawal on {current_date}. Skipping.")
                    continue

                # Check if there's enough gas stored for withdrawal
                if volume >= withdraw_vol:
                    volume -= withdraw_vol
                    total_sell_value += self.calculate_withdrawal_value(withdraw_vol, withdraw_price)
                    print(f"Withdrew {withdraw_vol} units of gas on {current_date} at price {withdraw_price:.2f}")
                else:
                    print(f"Insufficient stored volume on {current_date} for {withdraw_vol} units")

        # Calculate storage costs based on the entire duration of the contract
        contract_duration_days = (max(self.withdrawal_dates) - min(self.injection_dates)).days
        total_storage_cost = self.calculate_storage_cost(contract_duration_days)

        # Calculate the net contract value
        net_value = total_sell_value - total_buy_cost - total_storage_cost

        print(f"Total Buy Cost: {total_buy_cost:.2f}")
        print(f"Total Sell Value: {total_sell_value:.2f}")
        print(f"Total Storage Cost: {total_storage_cost:.2f}")
        print(f"Net Contract Value: {net_value:.2f}")

        return net_value

In [47]:
# Test Case
def test_natural_gas_storage_contract():
    # Define test input data
    injection_dates = ["2025-02-28", "2025-03-31", "2025-04-30"]
    injection_volumes = [1000, 1500, 1200] 
    withdrawal_dates = ["2025-07-31", "2025-08-31", "2025-09-30"]
    withdrawal_volumes = [800, 900, 1200]  # in units
    purchase_prices = [2.5, 3.0, 2.8]  # in currency per unit
    selling_prices = [3.5, 3.7]  # in currency per unit
    injection_withdrawal_rate = 0.1  # rate per unit (e.g., 10% fee)
    max_storage = 5000  # maximum storage capacity in units
    storage_costs_per_unit = 0.5  # cost to store one unit per month

    # Create instance of the contract class
    contract = NaturalGasStorageContract(
        injection_dates,
        injection_volumes,
        withdrawal_dates,
        withdrawal_volumes,
        purchase_prices,
        selling_prices,
        injection_withdrawal_rate,
        max_storage,
        storage_costs_per_unit
    )

    # Compute the net contract value
    net_value = contract.compute_net_contract_value()
    print("---------")
    print(f"Net Contract Value: ${net_value:.2f}")


# Run the case example
test_natural_gas_storage_contract()

Injected 1000 units of gas on 2025-02-28 at price 4.00
Injected 1500 units of gas on 2025-03-31 at price 5.00
Injected 1200 units of gas on 2025-04-30 at price 6.00
Withdrew 800 units of gas on 2025-07-31 at price 9.00
Withdrew 900 units of gas on 2025-08-31 at price 10.00
Withdrew 1200 units of gas on 2025-09-30 at price 11.00
Total Buy Cost: 19070.00
Total Sell Value: 29110.00
Total Storage Cost: 4.00
Net Contract Value: 10036.00
---------
Net Contract Value: $10036.00
